# Connection to our Neo4j server

In [2]:
from neo4j import __version__ as neo4j_version
print(neo4j_version)

4.3.6


In [3]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [4]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="s3cr3t")

In [4]:
query_string = '''
MATCH (n) RETURN n;
'''

conn.query(query_string)

[<Record n=<Node id=0 labels=frozenset({'Host'}) properties={'ip': '192.168.34.101'}>>,
 <Record n=<Node id=1 labels=frozenset({'Host'}) properties={'ip': '142.250.13.95'}>>,
 <Record n=<Node id=2 labels=frozenset({'Host'}) properties={'ip': '192.168.34.100'}>>,
 <Record n=<Node id=3 labels=frozenset({'Host'}) properties={'ip': '74.125.133.95'}>>,
 <Record n=<Node id=4 labels=frozenset({'Host'}) properties={'ip': '192.168.34.99'}>>,
 <Record n=<Node id=5 labels=frozenset({'Host'}) properties={'ip': '64.233.184.95'}>>]

Label nodes with internal and external

In [6]:
query_string ='''
MATCH (n:Host)
WITH n,
CASE 
WHEN n.ip STARTS WITH "192.168.34." THEN 'Internal'
ELSE 'External'
END AS type_connection
SET n.type_connection = type_connection
'''

conn.query(query_string)

[]

Label edges to distinguish the connection that are directed outside the internal network and outside the network

In [7]:
query_string ='''
MATCH ()-[r:TO]->(n:Host)
WITH n,r,
CASE 
WHEN n.type_connection='Internal' THEN 'Internal'
ELSE 'External'
END AS type_connection  
SET r.type_connection = type_connection
'''

conn.query(query_string)

[]